<h1>Install and import Library<h1>

In [ ]:
%pip install matplotlib
%pip install pandas openpyxl
%pip install seaborn
%pip install shapely
%pip install geodatasets
%pip install geopy

In [ ]:
# Dependencies
import pandas as pd
import geopandas as gpd
from pathlib import Path
import openpyxl
import zipfile
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import geodatasets
from geopy.geocoders import Nominatim

<h1> ETL - Load, transform and save data to SQL database <h1>

In [ ]:
# Upzip geojson file 
zip_file_path = 'Resources/California_Fire_Perimete.geojson.zip'

# Open the zip file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of the zip file
    zip_ref.extractall('Resources')

In [3]:
# Wildfire Data
# Name of the geojson file
file_js = Path('Resources/California_Fire_Perimete.geojson')

In [4]:
# Step 1: Load Wildfire GeoJSON Data
wildfire_gdf = gpd.read_file(file_js)

In [5]:
# Display the first few rows to understand the structure
print(wildfire_gdf.head())

   OBJECTID   YEAR_ STATE AGENCY UNIT_ID  FIRE_NAME   INC_NUM  \
0         1  2023.0    CA    CDF     SKU  WHITWORTH  00004808   
1         2  2023.0    CA    LRA     BTU     KAISER  00010225   
2         3  2023.0    CA    CDF     AEU    JACKSON  00017640   
3         4  2023.0    CA    CDF     AEU     CARBON  00018821   
4         5  2023.0    CA    CDF     AEU    LIBERTY  00018876   

                      ALARM_DATE                      CONT_DATE  CAUSE  \
0  Sat, 17 Jun 2023 00:00:00 GMT  Sat, 17 Jun 2023 00:00:00 GMT    5.0   
1  Fri, 02 Jun 2023 00:00:00 GMT  Fri, 02 Jun 2023 00:00:00 GMT    5.0   
2  Sat, 01 Jul 2023 00:00:00 GMT  Sun, 02 Jul 2023 00:00:00 GMT    2.0   
3  Tue, 11 Jul 2023 00:00:00 GMT  Tue, 11 Jul 2023 00:00:00 GMT    9.0   
4  Tue, 11 Jul 2023 00:00:00 GMT  Wed, 12 Jul 2023 00:00:00 GMT   14.0   

   C_METHOD  OBJECTIVE  GIS_ACRES COMMENTS COMPLEX_NAME  \
0       1.0        1.0   5.729125     None         None   
1       1.0        1.0  13.602380     None    

In [6]:
# Extract column names
column_names = wildfire_gdf.columns
print("Column Names:", column_names)

Column Names: Index(['OBJECTID', 'YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME',
       'INC_NUM', 'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'C_METHOD', 'OBJECTIVE',
       'GIS_ACRES', 'COMMENTS', 'COMPLEX_NAME', 'IRWINID', 'FIRE_NUM',
       'COMPLEX_ID', 'DECADES', 'geometry'],
      dtype='object')


In [7]:
# Check the minimum and maximum year in the dataset
min_year = wildfire_gdf['YEAR_'].min()
max_year = wildfire_gdf['YEAR_'].max()

print(f"Year Range: {min_year} to {max_year}")

Year Range: 1878.0 to 2023.0


In [10]:
# Step 2: Process Wildfire Data - Change column names
# Rename specific columns
wildfire_gdf = wildfire_gdf.rename(columns={
    'OBJECTID': 'ID',
    'YEAR_': 'Year',
    'STATE': 'State',
    'AGENCY': 'Agency',
    'UNIT_ID': 'Unit ID',
    'FIRE_NAME': 'Fire Name',
    'INC_NUM': 'Incident Number',
    'ALARM_DATE': 'Alarm Date',
    'CONT_DATE': 'Containment Date',
    'CAUSE': 'Cause',
    'C_METHOD': 'Collection Method',
    'OBJECTIVE': 'Management Objective',
    'GIS_ACRES': 'GIS Acres',
    'COMMENTS': 'Comments', 
    'COMPLEX_NAME': 'Complex Name',
    'IRWINID': 'IRWIN ID',
    'FIRE_NUM': 'Fire Number',
    'COMPLEX_ID': 'Complex ID',
    'DECADES':'Decades', 
    'geometry': 'Geometry'
})

In [11]:
#Confirm column name changes
column_names = wildfire_gdf.columns
print("Column Names:", column_names)

Column Names: Index(['ID', 'Year', 'State', 'Agency', 'Unit ID', 'Fire Name',
       'Incident Number', 'Alarm Date', 'Containment Date', 'Cause',
       'Collection Method', 'Management Objective', 'GIS Acres', 'Comments',
       'Complex Name', 'IRWIN ID', 'Fire Number', 'Complex ID', 'Decades',
       'Geometry'],
      dtype='object')


In [12]:
#Keep only a subset of columns for analysis
wildfire_gdf = wildfire_gdf[['ID', 'Year', 'State', 'Agency', 'Unit ID', 'Fire Name',
    'Incident Number', 'Alarm Date', 'Containment Date', 'Cause', 'GIS Acres', 
    'Comments','Complex Name', 'Fire Number', 'Decades','Geometry']]

In [ ]:
# Calculate the centroid of each geometry (for polygons)
wildfire_gdf['Centroid'] = wildfire_gdf['Geometry'].centroid

# Extract latitude and longitude from the centroid
wildfire_gdf['Latitude'] = wildfire_gdf['Centroid'].y
wildfire_gdf['Longitude'] = wildfire_gdf['Centroid'].x

# Convert the geometry column to WKT (Well-Known Text)
wildfire_gdf['Geometry'] = wildfire_gdf['Geometry'].apply(lambda x: x.wkt)

In [ ]:
#check transformed data
print(wildfire_gdf)

In [ ]:
#Check all columns
column_names = wildfire_gdf.columns
print("Column Names:", column_names)

In [ ]:
#check the sample size
num_records = len(wildfire_gdf)
print(f"Number of records: {num_records}")

In [ ]:
#check fire data type
print(wildfire_gdf.dtypes)

In [ ]:
#Transform Date columns to datetime
wildfire_gdf['Alarm Date'] = pd.to_datetime(wildfire_gdf['Alarm Date'], errors='coerce' )
wildfire_gdf['Containment Date'] = pd.to_datetime(wildfire_gdf['Containment Date'], errors='coerce' )

In [ ]:
#check how many dates are not valid
invalid_dates = wildfire_gdf[wildfire_gdf['Alarm Date'].isna()]
print(invalid_dates)

In [ ]:
#Extract Year Month info from Alarm Date
wildfire_gdf['DATE'] = wildfire_gdf['Alarm Date'].dt.to_period('M').dt.to_timestamp()

In [ ]:
#check fire date column
print(wildfire_gdf['DATE'])

In [ ]:
#Keep only a subset of columns for analysis - remove geometry and centroid data which SQL can't take
wildfire_gdf = wildfire_gdf[['ID', 'Year', 'State', 'Agency', 'Unit ID', 'Fire Name',
    'Incident Number', 'Alarm Date', 'Containment Date', 'Cause',
    'GIS Acres', 'Comments', 'Complex Name', 'Fire Number', 'Decades',
    'Latitude', 'Longitude']]

In [ ]:
#Connect to a SQL database
conn = sqlite3.connect('ca_wildfires.db')
c = conn.cursor()

In [ ]:
# Write the GeoDataFrame to the SQLite database
# The table name will be 'wildfires' in this example
wildfire_gdf.to_sql('wildfires', conn, if_exists='replace', index=False)

In [ ]:
# Commit the transaction and close the connection
conn.commit()
conn.close()

print("GeoJSON data with latitude and longitude loaded successfully into the SQLite database.")

In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect('ca_wildfires.db')
c = conn.cursor()

# Example query: Get the first 5 records
c.execute('SELECT * FROM wildfires LIMIT 5')
rows = c.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the connection
conn.close()

In [ ]:
# SQLite wildfire format 
#  TABLE "wildfires" CREATE TABLE "wildfires" (
# "ID" INTEGER,
#   "Year" REAL,
#   "State" TEXT,
#   "Agency" TEXT,
#   "Unit ID" TEXT,
#   "Fire Name" TEXT,
#   "Incident Number" TEXT,
#   "Alarm Date" TIMESTAMP,
#   "Containment Date" TIMESTAMP,
#   "Cause" REAL,
#   "GIS Acres" REAL,
#   "Comments" TEXT,
#   "Complex Name" TEXT,
#   "Fire Number" TEXT,
#   "Decades" REAL,
#   "Latitude" REAL,
#   "Longitude" REAL
# )

In [16]:
# Rainfall SF Data
# Name of the rainfall csv file
sf_rain= Path('Resources/sf_rainfall.csv')
# The correct encoding must be used to read the CSV in pandas
df_sf_rain = pd.read_csv(sf_rain)
# Preview of the rain fall dataFrame
df_sf_rain.head()

In [17]:
# Rainfall LA Data
# Name of the rainfall csv file
la_rain= Path('Resources/la_rainfall.csv')
# The correct encoding must be used to read the CSV in pandas
df_la_rain = pd.read_csv(la_rain)
# Preview of the rain fall dataFrame
df_la_rain.head()

In [18]:
# Rainfall San Diego Data
# Name of the rainfall csv file
sdg_rain= Path('Resources/sdg_rainfall.csv')
# The correct encoding must be used to read the CSV in pandas
df_sdg_rain = pd.read_csv(sdg_rain)
# Preview of the rain fall dataFrame
df_sdg_rain.head()

,Year,Precipitation (inches)
0,1896,26.42
1,1897,27.07
2,1898,17.29
3,1899,20.19
4,1900,24.48


In [ ]:
#Combine all rainfall data
df_rain = pd.concat([df_sf_rain, df_la_rain, df_sdg_rain], axis=0, ignore_index=True)
df_rain.head()

In [ ]:
#Transform date columns to the correct datetime format
df_rain['DATE TIME']=pd.to_datetime(df_rain['DATE TIME'])
df_rain['OBS DATE']=pd.to_datetime(df_rain['OBS DATE'])
df_rain.info()

In [ ]:
# Create a new column with just the year and month in datetime format
df_rain['YEAR MONTH'] = df_rain['DATE TIME'].dt.to_period('M').dt.to_timestamp()
df_rain.head()

In [ ]:
# Rainfall Station Geo Data
# Name of the rainfall station geo csv file
rainfall_station = Path('Resources/weather_station_geo.csv')
# The correct encoding must be used to read the CSV in pandas
df_rain_geo = pd.read_csv(rainfall_station)
# Preview of the rain fall dataFrame
df_rain_geo.head()

In [ ]:
#change Rainfall Station Geo Data to be consistent with df_rain data for merge later
df_rain_geo.columns = ['STATION NAME', 'STATION_ID', 'ELEV (FEET)', 'LATITUDE', 'LONGITUDE',
'COUNTY', 'OPERATOR AGENCY']

In [ ]:
#Inspect mergered data
df_precip = pd.merge(df_rain, df_rain_geo, on='STATION_ID', how = 'left')
df_precip.head()

In [ ]:
#Transform date columns to the correct datetime format
df_precip['DATE TIME']=pd.to_datetime(df_precip['DATE TIME'])
df_precip['OBS DATE']=pd.to_datetime(df_precip['OBS DATE'])
df_precip.info()

In [ ]:
# Create a new column with just the year and month in datetime format
df_precip['DATE'] = df_precip['DATE TIME'].dt.to_period('M').dt.to_timestamp()
df_precip.head()

In [ ]:
#Transform longtitude column from string to float
df_precip['LONGITUDE']=df_precip['LONGITUDE'].astype('float')
df_precip.info()

In [ ]:
# Average Temperatrue Data
sf_temp_path = Path('Resources/avg-temps-sf.csv')
la_temp_path = Path('Resources/avg-temps-la.csv')
sd_temp_path = Path('Resources/avg-temps-sd.csv')
sac_temp_path = Path('Resources/avg-temps-sac.csv')
bf_temp_path = Path('Resources/avg-temps-bf.csv')
erk_temp_path = Path('Resources/avg-temps-erk.csv')
ca_temp_path = Path('Resources/avg-temps-ca.csv')
dv_temp_path = Path('Resources/avg-temps-dv.csv')
fr_temp_path = Path('Resources/avg-temps-fr.csv')
mo_temp_path = Path('Resources/avg-temps-mo.csv')

In [ ]:
# Read in CSVs without unnecessary rows
sf_temps_df = pd.read_csv(sf_temp_path, skiprows=3)
la_temps_df = pd.read_csv(la_temp_path, skiprows=3)
sd_temps_df = pd.read_csv(sd_temp_path, skiprows=3)
sac_temps_df = pd.read_csv(sac_temp_path, skiprows=3)
bf_temps_df = pd.read_csv(bf_temp_path, skiprows=3)
erk_temps_df = pd.read_csv(erk_temp_path, skiprows=3)
ca_temps_df = pd.read_csv(ca_temp_path, skiprows=3)
dv_temps_df = pd.read_csv(dv_temp_path, skiprows=3)
fr_temps_df = pd.read_csv(fr_temp_path, skiprows=3)
mo_temps_df = pd.read_csv(mo_temp_path, skiprows=3)

In [ ]:
# Create list of all city dfs
cities_temps = [sf_temps_df, la_temps_df, sd_temps_df, sac_temps_df, bf_temps_df, erk_temps_df, dv_temps_df, fr_temps_df, mo_temps_df, ca_temps_df]

In [ ]:
# Convert date to datetime format and split year and month into two columns, then drop 'Date' column
for city in cities_temps:
    city['Date'] = pd.to_datetime(city['Date'], format='%Y%m')
    city['Month'] = city['Date'].dt.month
    city['Year'] = city['Date'].dt.year
    city = city.drop('Date', axis=1, inplace=True)

In [ ]:
# Rename each 'Value' column to 'Temperatrue'
sf_temps_df = sf_temps_df.rename(columns={'Value': 'Temperatrue'})
la_temps_df = la_temps_df.rename(columns={'Value': 'Temperatrue'})     
sd_temps_df = sd_temps_df.rename(columns={'Value': 'Temperatrue'})     
sac_temps_df = sac_temps_df.rename(columns={'Value': 'Temperatrue'})     
bf_temps_df = bf_temps_df.rename(columns={'Value': 'Temperatrue'})     
erk_temps_df = erk_temps_df.rename(columns={'Value': 'Temperatrue'}) 
ca_temps_df = ca_temps_df.rename(columns={'Value': 'Temperatrue'})  
dv_temps_df = dv_temps_df.rename(columns={'Value': 'Temperatrue'})     
fr_temps_df = fr_temps_df.rename(columns={'Value': 'Temperatrue'}) 
mo_temps_df = mo_temps_df.rename(columns={'Value': 'Temperatrue'})  

In [ ]:
# Reorder each df
sf_temps_df = sf_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
la_temps_df = la_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
sd_temps_df = sd_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
sac_temps_df = sac_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
bf_temps_df = bf_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
erk_temps_df = erk_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
ca_temps_df = ca_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
dv_temps_df = dv_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
fr_temps_df = fr_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]
mo_temps_df = mo_temps_df[['Date', 'Year', 'Month', 'Temperatrue']]

In [ ]:
# Check each df
print(sf_temps_df)
print(la_temps_df)
print(sd_temps_df)
print(sac_temps_df)
print(bf_temps_df)
print(erk_temps_df)
print(ca_temps_df)
print(dv_temps_df)
print(fr_temps_df)
print(mo_temps_df)

In [ ]:
sf_temps_df.insert(0, 'City', 'San Francisco')
la_temps_df.insert(0, 'City', 'Los Angeles')
sd_temps_df.insert(0, 'City', 'San Diego')
sac_temps_df.insert(0, 'City', 'Sacramento')
bf_temps_df.insert(0, 'City', 'Bakersfield')
erk_temps_df.insert(0, 'City', 'Eureka')
ca_temps_df.insert(0, 'City', 'California')
dv_temps_df.insert(0, 'City', 'Death Valley')
fr_temps_df.insert(0, 'City', 'Fresno')
mo_temps_df.insert(0, 'City', 'Modesto')

In [ ]:
# Create a function to get the coordinates for a city:
def get_coordinates(city_name):
    geolocator = Nominatim(user_agent="wildires")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

In [ ]:
cities_temps = [sf_temps_df, la_temps_df, sd_temps_df, sac_temps_df, bf_temps_df, erk_temps_df, dv_temps_df, fr_temps_df, mo_temps_df]
cities_temps_df = pd.concat(cities_temps, axis=0)

In [ ]:
cities = ['San Francisco', 'Los Angeles', 'San Diego', 'Sacramento', 'Bakersfield', 'Eureka', 'California', 'Death Valley', 'Fresno', 'Modesto']
cities_coords_df = pd.DataFrame({'City': cities})
cities_coords_df

In [ ]:
cities_coords_df["Latitude"], cities_coords_df["Longitude"] = zip(*cities_coords_df["City"].apply(get_coordinates))

In [ ]:
cities_merged_df = pd.merge(cities_temps_df, cities_coords_df, on='City', how='left')
cities_merged_df

<h1>Visualization</h1>

In [ ]:
#Number of Wildfires per Year from 1940 to 2022
#Objective: Visualize the trend of wildfire occurrences over the years.
#Visualization: Line chart showing the number of wildfires per year.

# Connect to the SQLite database
conn = sqlite3.connect('ca_wildfires.db')

# Query the database to count the number of wildfires per year
query = "SELECT Year, COUNT(*) as Number_of_Wildfires FROM wildfires WHERE Year >= 1940 AND Year <= 2022 GROUP BY Year ORDER BY Year"
wildfire_counts = pd.read_sql(query, conn)

# Plot the number of wildfires per year
plt.figure(figsize=(10, 6))
plt.plot(wildfire_counts['Year'], wildfire_counts['Number_of_Wildfires'], marker='o')
plt.title('Number of Wildfires per Year in California')
plt.xlabel('Year')
plt.ylabel('Number of Wildfires')
plt.grid(True)
plt.show()

# Save the plot to a file  in image folder
plt.savefig('Images/wildfires_per_year.png', dpi=300, bbox_inches='tight')

In [ ]:
#Total Acres Burned per Year from 1940 to 2022
# Objective: Show the total area burned by wildfires each year.
# Visualization: Bar chart displaying the total acres burned annually.
# Connect to the SQLite database
# Connect to the SQLite database
conn = sqlite3.connect('ca_wildfires.db')

# Query the database to sum the acres burned per year (focus on 1940 to 2020)
query = 'SELECT Year, SUM("GIS Acres") as Total_Acres_Burned FROM wildfires WHERE Year >= 1940 AND Year <= 2022 GROUP BY Year ORDER BY Year'
acres_burned = pd.read_sql(query, conn)

# Convert Year to integer
acres_burned['Year'] = acres_burned['Year'].astype(int)

# Plot the total acres burned per year
plt.figure(figsize=(16, 8))  # Increase figure size for better readability
sns.barplot(x='Year', y='Total_Acres_Burned', data=acres_burned, palette='OrRd')
plt.title('Total Acres Burned per Year in California (1940-2020)')
plt.xlabel('Year')
plt.ylabel('Total Acres Burned')

# Rotate all x-axis labels to avoid overlap
plt.xticks(rotation=45, fontsize=6)

plt.show()

# Close the database connection
conn.close()

# Save the plot to a file in image folder
plt.savefig('Images/total_acres_burned_per_year.png', dpi=300, bbox_inches='tight')

In [ ]:
#Heatmap of Wildfire Occurrences by Month and Year
#Objective: Understand seasonal trends in wildfire occurrences.
#Visualization: Heatmap showing the number of wildfires by month and year.
# Connect to the SQLite database
conn = sqlite3.connect('ca_wildfires.db')

# Query the database to get the month and year of each fire
query = """
SELECT strftime('%Y', "Alarm Date") as Year, strftime('%m', "Alarm Date") as Month, COUNT(*) as Number_of_Wildfires 
FROM wildfires 
WHERE CAST(strftime('%Y', "Alarm Date") AS INTEGER) >= 1940 
AND CAST(strftime('%Y', "Alarm Date") AS INTEGER) <= 2022 
GROUP BY Year, Month 
ORDER BY Year, Month
"""
monthly_counts = pd.read_sql(query, conn)

# Ensure the Month and Year columns are strings
monthly_counts['Month'] = monthly_counts['Month'].astype(str)
monthly_counts['Year'] = monthly_counts['Year'].astype(str)

# Aggregate the data to ensure there are no duplicate Month-Year combinations
monthly_counts_agg = monthly_counts.groupby(['Month', 'Year'], as_index=False).sum()

# Pivot the data for heatmap
monthly_counts_pivot = monthly_counts_agg.pivot(index='Month', columns='Year', values='Number_of_Wildfires')

# Replace NaN values with 0 (for months/years with no wildfires)
monthly_counts_pivot = monthly_counts_pivot.fillna(0)

# Plot a heatmap of wildfire occurrences by month and year
plt.figure(figsize=(18, 12))
sns.heatmap(monthly_counts_pivot, cmap='Reds', annot=True, annot_kws={"size": 6})
plt.title('Heatmap of Wildfire Occurrences by Month and Year (1940-2022)')
plt.xlabel('Year')
plt.ylabel('Month')
# Rotate all x-axis labels to avoid overlap
plt.xticks(rotation=45, fontsize=6)
plt.show()

# Close the database connection
conn.close()

# Save the plot to a file
plt.savefig('Images/wildfire_heatmap_month_year.png', dpi=300, bbox_inches='tight')

In [ ]:
#Wildfire Occurrences and Acreage Burned
# Load data from the SQLite database
conn = sqlite3.connect('ca_wildfires.db')
wildfire_data = pd.read_sql('SELECT Year, COUNT(*) as num_fires, SUM("GIS Acres") as total_acres FROM wildfires GROUP BY Year', conn)
conn.close()

# Plot wildfire occurrences and acreage burned
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Year')
ax1.set_ylabel('Number of Fires', color='tab:orange')
ax1.plot(wildfire_data['Year'], wildfire_data['num_fires'], color='tab:orange', label='Number of Fires')
ax1.tick_params(axis='y', labelcolor='tab:orange')

ax2 = ax1.twinx()
ax2.set_ylabel('Total Acres Burned', color='tab:red')
ax2.plot(wildfire_data['Year'], wildfire_data['total_acres'], color='tab:red', label='Total Acres Burned')
ax2.tick_params(axis='y', labelcolor='tab:red')

fig.tight_layout()
plt.title('Wildfire Occurrences and Total Acres Burned in California Over Time')
plt.show()

# Save the plot to a file
plt.savefig('Images/wildfire_occurence_acres_over_time.png', dpi=300, bbox_inches='tight')